# Homework 1
# Natural Language Processing, CSCI-SHU376 Spring 2024

## Due Feb 25, 2024 at 11:59pm China Time

Name: Yufeng Xu

NetID: yx3038

Please submit the following items to Gradescope:
* Your Colab notebook link (by clicking the Share button at the top-right corner of the Colab notebook, share to anyone).
* The printout of your run in Colab notebook in pdf format

Note:
* late submission is allowed, following our 72hr policy.
* All solutions must be from your own work.
* Total points of the assignment is 100.

# Setup environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# This makes your Google drive as the HOME directory in Colab
%env HOME=/content/drive/My Drive/
# This folder will be used to store your models and results
!mkdir -p ~/Courses/nlp_spring2024/homework1/
%cd ~/Courses/nlp_spring2024/homework1

env: HOME=/content/drive/My Drive/
/content/drive/My Drive/Courses/nlp_spring2024/homework1


In [1]:
!pip install nltk datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.1/27.1 MB 4.4 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=

# 1. N-gram LM with ADD-one smoothing [50 pts]

Add-one smoothing is a method to smooth the N-gram counts. Please refer to the lecture slides for details.

##__a__. Implement `class ADD1LM`. [50 pts]

In this question, you will implement ADD-ONE LM.

The ADD-ONE LM `class ADD1LM` support the following functions:
* Select the top-K vocabulary based on the `min_count` on the training data. If `min_count` is 1, all vocabulary will be selected from the training data.
* Calculate raw N-gram counts from a dataset.
* Estimate LM model parameters: with add-one smoothing
* Calculate word perplexity on a dataset.
* Generate random sentences using the LM.

All word tokens that are not in the selected vocabulary should be mapped to `<unk>`, the unknown word token.

### Dataset
We use the datasets module from Huggingface to load the Penntree bank text data for LM experiment. Example of getting an iterable over sentences is shown in `class ADD1LM`.

### Limited usage of NLTK
We allow using `nltk.ngrams` to compute the ngrams given a list of word tokens. But you are NOT allowed to use any other functions and data structures in nltk for this assignment.

### Data structure
Please use the provided data structure to complete the task. We assume using the Python dictionaries to store the discounted probs and backoff weights, as provided in `class ADD1LM`.

Context: A tuple of strings

word: A string

order 1: unigram, order 2: bigram and so forth.

For example:

`self._prob[1][context=()][word="is"]` stores the probs P*("is") for unigram (N-gram order = 1).

`self._prob[2][context=("this",)][word="is"]` stores the probs probs P*("is"|"this") for bigrams (N-gram order = 2).

`self._prob[3][context=("this","is",)][word="a"]` stores the probs P*("a"|"this is") for trigrams (N-gram order = 3).

We also provide a data structure to store the N-gram counts in `self.counters`.

Alternatively, you may also use other Python data structure as you see fit to store the LM parameters.

### Correctness
All the estimated probabilities of *any* context must sum to 1.0. You may use `def check_lm()` and `def check_prob()` to check the correctness by trying out some word context at different N-gram orders. You should carefully check the validity of your LM.

In [11]:
import nltk
import math
import numpy as np
import random
import datasets
import multiprocessing
from tqdm import tqdm
from datasets import load_dataset
from collections import defaultdict

nltk.download('punkt')

class SimpleTokenizer:
  def __init__(self):
    pass

  def tokenize(self, sentence): ####### the implementation of tokenizer
    return sentence.strip().split()

class ADD1LM:
  def __init__(self, order, min_count, tokenizer):
    """
    order: integer. order = 1...N
    min_count: at least 1
    tokenizer: tokenize a given sentence strin
    """
    self._bos = "<s>" # begin of sentence
    self._eos = "</s>" # end of sentence
    self._unk = "<unk>" # word not in vocabulary

    # set class variables
    self._order = order
    self._min_count = min_count
    self._tokenizer = tokenizer

    # data structures for the LM
    # prob
    self._prob = [defaultdict(lambda: defaultdict(float)) for i in range(order+1)]

    # data structure to store the N-gram counts
    self._counters = [defaultdict(lambda: defaultdict(float)) for i in range(order+1)]

    # vocabulary set
    self._vocab = set()

  def print_counter(self, counter):
    """
    Debugging purpose: You may want to print the counters based on only a handful of sentences
    and see if the computed counts are equal to the ones computed by hand.
    """
    for context in counter.keys():
      for word in counter[context].keys():
        print("{} {} {}".format(context, word, counter[context][word]))

  def _select_vocab(self, dataset, min_count):
    """
    """
    print('Selecting vocabulary...')
    counter = defaultdict(int)
    for sample in dataset:
      tokens = self._tokenizer.tokenize(sample['sentence'])

    # TODO: Your code
      for word in tokens:
        counter[word] += 1
        
      for word in counter.keys():
        if counter[word] >= min_count:
          self._vocab.add(word.lower())
    self._vocab.add(self._bos)
    self._vocab.add(self._eos)
    #print(self._bos in self._vocab)
    #print(self._eos in self._vocab)
    #print(self._unk in self._vocab)
      
    print("Size of vocabulary is {}".format(len(self._vocab)))

  def compute_count(self, dataset, order):
    """
    dataset: iterable yielding a dictionary object that has a field 'sentence'
    order: integer. order = 1...N
    counter is just a dictionary of a dictionary of a integer
    counter[context][word] stores the raw N-gram counts
    """
    counter = defaultdict(lambda: defaultdict(int))

    for sample in dataset:
      tokens = self._tokenizer.tokenize(sample['sentence'])

      # map word not in vocab to <unk>
      # TODO: Your code
      for i in range(len(tokens)):
        if tokens[i] not in self._vocab:
          tokens[i] = self._unk
      
      # a generator of N-gram tuples
      text_ngrams = nltk.ngrams(tokens, n=order, pad_left=True, pad_right=True, left_pad_symbol=self._bos, right_pad_symbol=self._eos)
      for each in list(text_ngrams):
        counter[each[:-1]][each[-1]] += 1

    return counter

  def compute_prob(self, context, word, order):
    """
    context: tuple of string
    word: string
    """
    p = 0.0
    beta = 1
    # TODO: Your code
    if order > 1 and context != (self._bos,):
      p = (self._counters[order][context][word] + beta) / (self._counters[order - 1][context[:-1]][context[-1]] + len(self._vocab) * beta)
    else:
      p = (self._counters[order][context][word] + beta) / (sum(self._counters[order][context].values()) + len(self._vocab) * beta)
    #print(f'Context: {context} Word: {word} Prob: {p}')
    return p

  def train(self, dataset):
    """
    Train a LM.
    dataset: an iterable object
    """
    # select vocab by taking the top-K
    print("estimating vocabulary set with mincount = {}".format(self._min_count))
    self._select_vocab(dataset, self._min_count)

    # compute the raw count of the highest N-order
    #print("Compute raw count for order {}".format(self._order))
    #self._counters[self._order] = self.compute_count(dataset, self._order)

    # TODO: Your Code:
    # Use the member functions to complete the training steps
    for o in range(1, self._order + 1):

      print(f'Computing raw count for order {o}...')
      self._counters[o] = self.compute_count(dataset, o)

      print(f'Computing probability for order {o}...')
      for context in tqdm(self._counters[o].keys()):
        for word in self._vocab:
          self._prob[o][context][word] = self.compute_prob(context, word, o)

  def perplexity(self, dataset):
    """
    Given a test file that contains a list of sentences line-by-line, read all lines and
    compute word perplexity of the test file.
    Please do not change the code here as it is for evaluation for all students.
    """
    log_prob = 0.0
    N = 0.0
    for sample in dataset:
      tokens = self._tokenizer.tokenize(sample['sentence'])

      # map word not in vocab to <unk>
      tokens = [w if w in self._vocab else self._unk for w in tokens]

      # N-grams of sentence
      text_ngrams = nltk.ngrams(tokens, n=self._order, pad_left=True, pad_right=True, left_pad_symbol=self._bos, right_pad_symbol=self._eos)

      # x is a tuple of N-gram tokens
      for x in text_ngrams:
        context, word = x[:-1], x[-1]
        p = self.compute_prob(context, word, self._order)
        if p < 1e-10:
          print("ERROR: small prob for {} {} {}".format(" ".join(context), word, p))
        else:
          log_prob += math.log(p)
          N += 1
    
    return math.exp(-log_prob / N)

  def generate(self):
    """
    Generate a random sentence using the trained LM.
    Return a sentence.
    """

    context = [self._bos] * (self._order-1)
    result = []

    # TODO: Your code
    # Think about how each token is generated from a LM given a context. This involves
    # sampling a word from your LM.
    while 1:
      candidates = self._prob[self._order][tuple(context)]
      k = 4
      top_k = sorted(candidates, key = lambda x : self._prob[self._order][tuple(context)][x], reverse = True)[:k]
      #p = [self._prob[self._order][tuple(context)][x] for x in top_k]
      #print(top_5)
      #print(p)
      choice = random.choice(top_k)
      while choice == self._unk:
        choice = random.choice(top_k)
      if choice == self._eos:
        break
      result.append(choice)
      context = context[1:] + [choice]
    
    return " ".join(result)

  def check_prob(self, context, order):
    """
    This is for checking whether your LM behaves properly on each word context.
    """
    z = 0.0
    for word in self._vocab:
      if word != self._bos:
        z += self.compute_prob(context, word, order)

    epsilon = 1e-4
    if z < 1 - epsilon or z > 1 + epsilon:
      print(z)
      print(sum(self._counters[order][context].values()))
      print(self._counters[order - 1][context[:-1]][context[-1]])
      print("Prob sum check failed at order {} for context {}!".format(order, context))
      return False
    return True

  def check_lm(self):
    """
    This is for checking whether your LM behaves properly on each word context.
    """
    # check 1-gram prob, should sum to 1!
    #self.check_prob((), 1)

    # check high-order prob
    for o in range(1, self._order+1):
      print(f'Checking probability for order {o}...')
      for i, context in enumerate(self._prob[o].keys()):
        self.check_prob(context, o)
        if i >= 1000: break

[nltk_data] Downloading package punkt to /Users/zephyr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Main *program*

In [12]:
# print the timestamp (Please do not remove)
!date

if __name__ == '__main__':
    dataset = load_dataset("ptb_text_only")
    lm = ADD1LM(order=2, min_count=1, tokenizer=SimpleTokenizer())
    lm.train(dataset['train'])

    # check probs for all orders. should sum to 1.0 in any context
    lm.check_lm()

    train_ppl = lm.perplexity(dataset['train'])
    valid_ppl = lm.perplexity(dataset['validation'])
    test_ppl = lm.perplexity(dataset['test'])

    print("Training perplexity = {}".format(train_ppl)) # Expect: ~78
    print("Validation perplexity = {}".format(valid_ppl)) # Expect: ~208
    print("Test perplexity = {}".format(test_ppl)) # Expect: ~192

Sun Feb 25 12:01:03 CST 2024
estimating vocabulary set with mincount = 1
Selecting vocabulary...
Size of vocabulary is 10001
Computing raw count for order 1...
Computing probability for order 1...


100%|██████████| 1/1 [00:01<00:00,  1.17s/it]


Computing raw count for order 2...
Computing probability for order 2...


100%|██████████| 9999/9999 [01:16<00:00, 130.61it/s]


Checking probability for order 1...
Checking probability for order 2...
Training perplexity = 677.5237127695859
Validation perplexity = 845.9791270087259
Test perplexity = 813.9470974138362


In [13]:
# look at some sentence
print(dataset['train'])
print(dataset['train'].features)
print(dataset['train'].num_rows)
print(dataset['train'][0])
print(dataset['train'][100])
print(dataset['train'][200])

Dataset({
    features: ['sentence'],
    num_rows: 42068
})
{'sentence': Value(dtype='string', id=None)}
42068
{'sentence': 'aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter'}
{'sentence': "plans that give advertisers discounts for maintaining or increasing ad spending have become permanent <unk> at the news <unk> and underscore the fierce competition between newsweek time warner inc. 's time magazine and <unk> b. <unk> 's u.s. news & world report"}
{'sentence': 'at cray computer he will be paid $ N'}


##__d__. Use your model to generate 10 sentences. Can you comment on the

*   List item
*   List item

quality of the generated sentences? [5 pts]

In [14]:
# print the timestamp (Please do not remove)
!date

# TODO: Your code
for i in range(10):
    res = lm.generate()
    print(res.capitalize())

Sun Feb 25 12:03:56 CST 2024
In new york stock
The new company is the u.s.
The u.s.
In a share from the company is n't be able gradually a share in a $ sharing to the company 's largest in new company is a $ finished lower
In new york
But they are the company 's a share in the u.s. and the new jersey and the company 's largest cable operator 's stock
But the company is the new company is a year earlier this month the new company 's a $ 300-a-share $ 300-a-share buy-out
The company 's a share from $ sharing of $ sharing a share from $ finished with a year ago and a year ago when he says mr. noriega was the new york and a share in a share a share in a share
But he says
But the new jersey 's stock market for a year ago


**Your answer**: 

The generated sentences demonstrate good diversity as I used top-3 sampling, but the fluency is poor because we only used a 2-gram model, which has poor representation of the larger semantic context.

Also, I excluded the unknown tokens when generating the sentences to reduce ambiguity of the result.

# 2. Multinomial Logistic Regression [50 pts]
In this question, we will focus on intent classification using the banking corpora which are available via the Huggingface dataset library.
This dataset is composed of 10003 training samples and 3080
test samples. There are 77 intents in the data. You can view the data from the provided codes below.

In Part I of the exercise, we provided code that loads the dataset using `datasets.load_dataset` and
divides the original training data into our training set and validation set in the ratio of 9:1, so that you can use the validation set for hyper-parameter model tuning. As always, you should not tune on the test set for fair reporting of test results.

## Setup environment

In [15]:
!pip install datasets nltk scikit-learn

In [2]:
import numpy as np
import os
import torch
from collections import Counter
import operator
from torch.utils.data import Dataset
import torch.nn as nn
from torch.autograd import Variable
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
english_stopwords = stopwords.words('english')

[nltk_data] Downloading package punkt to /Users/zephyr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zephyr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Some of the Hyper parameters

In [78]:
learning_rate = 0.1
vocab_size = 64000 # number N-gram features in the vocabulary base
num_epochs = 100 # number epoch to train
batch_size = 16
ngram_n = 1 # the n in n-gram
num_class = 77 # Number of intents in banking77 corpus

##Part I: Dataset

In [80]:
#Data Loading from Huggingface
import datasets
from datasets import load_dataset
data = load_dataset("banking77")

In [ ]:
data.keys()

dict_keys(['train', 'test'])

In [ ]:
print(data['train'])
print(data['test'])
label_dist = [0] * num_class
for k in data['train']['label']:
  label_dist[k] += 1
print(label_dist)

Dataset({
    features: ['text', 'label'],
    num_rows: 10003
})
Dataset({
    features: ['text', 'label'],
    num_rows: 3080
})
[159, 110, 126, 87, 127, 171, 181, 156, 157, 129, 59, 153, 112, 139, 112, 187, 168, 167, 61, 177, 160, 122, 86, 35, 129, 153, 173, 133, 182, 121, 121, 121, 112, 118, 166, 137, 126, 97, 106, 129, 98, 82, 121, 120, 105, 159, 143, 149, 148, 115, 95, 162, 169, 161, 129, 108, 111, 114, 114, 145, 97, 146, 103, 175, 172, 113, 171, 128, 102, 104, 113, 126, 41, 135, 121, 180, 163]


In [ ]:
print(data['train'][:3])

{'text': ['I am still waiting on my card?', "What can I do if my card still hasn't arrived after 2 weeks?", 'I have been waiting over a week. Is the card still coming?'], 'label': [11, 11, 11]}


In [ ]:
TRAIN_SIZE = int(len(data["train"]) * 0.9)
VALIDATION_SIZE = len(data["train"]) - TRAIN_SIZE
TEST_SIZE = len(data["test"])
PADDING_IDX = 0
print(TRAIN_SIZE)
print(VALIDATION_SIZE)
print(TEST_SIZE)
print(english_stopwords)

9002
1001
3080
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 

In [ ]:
class Datum():
    """
    Class that represents a train/validation/test datum
    - self.raw_text
    - self.label
    - self.tokens: list of tokens
    - self.token_idx: index of each token in the text
    """
    def __init__(self, raw_text, label):
        self.raw_text = raw_text
        self.label = label

    def set_ngram(self, ngram_ctr):
        self.ngram = ngram_ctr

    def set_token_idx(self, token_idx):
        self.token_idx = token_idx

    def set_tokens(self, tokens):
        self.tokens = tokens

def preprocess_text(text):
    """
    Function that cleans the string
    """
    text = text.lower().replace("<br />", " ")
    return text

def create_dataset(data):
  output = []
  for i in range(len(data['label'])):
    output.append(Datum(preprocess_text(data['text'][i]),data['label'][i]))
  return output

In [ ]:
shuffle = np.random.RandomState(seed=42).permutation(TRAIN_SIZE + VALIDATION_SIZE)
TRAIN_ID = shuffle[:TRAIN_SIZE]
VAL_ID = shuffle[TRAIN_SIZE:]

In [ ]:
trainset = create_dataset(data['train'][TRAIN_ID])
valset = create_dataset(data['train'][VAL_ID])
testset = create_dataset(data['test'][:TEST_SIZE])

In [ ]:
from collections import defaultdict

# dump label counts for train set
train_label_cnt = defaultdict(int)
for i, sample in enumerate(trainset):
  x,y = sample.raw_text, sample.label
  if i < 2:
    print(x, y)
  train_label_cnt[y] += 1
print(train_label_cnt, len(trainset))

is it possible for me to change my pin number? 21
i'm not sure why my card didn't work 25
defaultdict(<class 'int'>, {21: 101, 25: 133, 59: 134, 15: 171, 5: 157, 27: 116, 34: 150, 53: 144, 7: 139, 17: 153, 57: 105, 60: 94, 24: 119, 6: 158, 69: 94, 12: 98, 4: 116, 39: 116, 74: 111, 54: 114, 76: 152, 47: 132, 19: 152, 48: 135, 32: 105, 29: 111, 13: 124, 45: 140, 20: 145, 52: 150, 11: 140, 38: 98, 73: 116, 28: 162, 30: 109, 70: 100, 75: 165, 63: 160, 40: 91, 64: 153, 18: 51, 66: 147, 35: 123, 56: 97, 61: 139, 46: 124, 3: 82, 23: 31, 16: 150, 51: 145, 37: 89, 31: 113, 72: 37, 58: 96, 33: 115, 9: 122, 67: 117, 2: 113, 8: 141, 50: 81, 65: 100, 71: 115, 43: 105, 1: 96, 41: 76, 26: 158, 55: 97, 14: 100, 0: 143, 22: 75, 68: 96, 49: 104, 36: 116, 62: 93, 42: 108, 44: 92, 10: 52}) 9002


In [ ]:
# dump label counts for val set
val_label_cnt = defaultdict(int)
for i, sample in enumerate(valset):
  x,y = sample.raw_text, sample.label
  if i < 2:
    print(x, y)
  val_label_cnt[y] += 1
print(val_label_cnt, len(valset))

there is a payment with my card which i definitely did not make by me .never seen that it before. 16
can i see where my money came from? 70
defaultdict(<class 'int'>, {16: 18, 70: 13, 59: 11, 24: 10, 43: 15, 58: 18, 1: 14, 17: 14, 5: 14, 73: 19, 63: 15, 37: 8, 7: 17, 66: 24, 26: 15, 28: 20, 38: 8, 56: 14, 19: 25, 54: 15, 40: 7, 6: 23, 69: 10, 23: 4, 74: 10, 64: 19, 2: 13, 45: 19, 48: 13, 20: 15, 0: 16, 11: 13, 25: 20, 29: 10, 46: 19, 71: 11, 36: 10, 75: 15, 32: 7, 55: 11, 51: 17, 8: 16, 21: 21, 15: 16, 30: 12, 61: 7, 39: 13, 3: 5, 60: 3, 52: 19, 4: 11, 65: 13, 27: 17, 47: 17, 14: 12, 12: 14, 34: 16, 35: 14, 44: 13, 31: 8, 42: 13, 13: 15, 57: 9, 18: 10, 62: 10, 72: 4, 53: 17, 49: 11, 9: 7, 76: 11, 67: 11, 68: 6, 50: 14, 22: 11, 33: 3, 41: 6, 10: 7}) 1001


## Part II: Feature Engineering - Bag of N-gram [25 pts]
The first step to deal with text data is to transform them into logic structure that
computers can "understand". For this problem, we will
use the trick of Bag of N-grams. A (word-level) N-gram can be thought of as a
continuous sequence of tokens in a document. For instance, given a document "I
love NLP", if we tokenize it using space, we will have unigrams {I; love; NLP},
a bigrams {(I; love); (love; NLP )}, and trigram {(I love nlp)}.

To represent N-gram, we will first set a vocabulary base V which can be
viewed as a set of words. Since we are limited by the available computational
resources, we need to set a limit on the number of vocabularies we add to our
vocabulary base. A common way to construct vocabulary base is to choose the
top k (a hyper-parameter set by user) most frequently occurred N-grams in the
data set.

We provide you with a top-level function
process text dataset that transforms a collection of text data points into a collection of n-gram indices. The n-gram indices are served as a sparse vector of an input document.
Your job is to fill in the code for the
helper functions that complete this process.
You will find the Python Counter object very helpful in this part of the assignment.

Please refer to https://docs.python.org/2/library/collections.html#collections.Counter for more info.

2.1 Extract N-gram from Text

Function `def extract_ngram_from_text` takes two inputs: a raw text string text and
an integer n which represents the max length of continuous tokens we would like
to extract from text. For input ‘I love NLP’ and n = 2, this function should extract all unigram and bigrams tokens = {I; love; NLP; (I; love); (love; NLP )}.
Note that in addition to tokens, this program should also output an Python
Counter object that counts the number of occurrences for each N-gram in
tokens. Please fill in your code to complete this function.

2.2 Construct Vocabulary Base

Once we obtain the N-grams for each sample, we can construct our vocabulary
base. Function `def construct_ngram_indexer` takes in two arguments ngram counter list,
which is a collection of Python Counter mentioned in the previous problem, and
an integer topk. This function should form a vocabulary base using the most
common topk N-grams. Moreover, we would like to take a further step and
create a dictionary ngram indexer that maps each N-gram in our vocabulary
base to a unique integer that represents the N-gram’s identity. Note that index
0 is reserved for special padding symbol which will be explained later. Please
fill in your code to complete this function.

2.3 Map N-gram to Index

Lastly, function token to index takes in tokens, which is a list of N-grams,
and ngram indexer constructed above and maps each N-gram in the list to its
corresponding index. Please fill in your code to complete this function.

In [ ]:
def extract_ngram_from_text(text, n, remove_stopwords=True):
    """
    Function that retrieves all n-grams from the input string
    @param text: raw string
    @param n: integer that tells the model to retrieve all k-gram where k<=n
    @return ngram_counter: a counter that maps n-gram to its frequency
    @return tokens: a list of parsed ngrams
    """
    # tokenize words - for simplicity just split by space
    #tokens = text.split(" ")
    tokens = nltk.word_tokenize(text)
    if remove_stopwords:
        tokens = [token for token in tokens if token not in english_stopwords]
    # extract n grams
    # TODO: replace with your code
    ngram_counter = Counter()
    all_ngrams = set()
    for o in range(1, n + 1):
        text_ngrams = nltk.ngrams(tokens, n=o, pad_left=False, pad_right=False)
        for ngram in text_ngrams:
            ngram_counter[ngram] += 1
            all_ngrams.add(ngram)
    #all_ngrams = ["this", "is", "an", "ngram"]
    all_ngrams = list(all_ngrams)
    #print(all_ngrams)
    return ngram_counter, all_ngrams


def construct_ngram_indexer(ngram_counter_list, topk):
    """
    Function that selects the most common topk ngrams
    @param ngram_counter_list: list of counters
    @param topk, int: # of
    @return ngram2idx: a dictionary that maps ngram to an unique index
    """
    # TODO: fill in your code here
    # find the top k ngram
    # maps the ngram to an unique index
    ngram_indexer = {}
    tot_counter = Counter()
    vocab = set()
    i = 1
    for counter in ngram_counter_list:
        for ngram in counter:
            tot_counter[ngram] += counter[ngram]
        
    for ngram in tot_counter.keys():
        vocab.add(ngram)
    ngram_indexer = {ngram : i + 1 for i, ngram in enumerate(vocab)}
    #print(ngram_indexer)
    return ngram_indexer

def token_to_index(tokens, ngram_indexer):
    """
    Function that transform a list of tokens to a list of token index.
    @param tokens: list of ngram
    @param ngram_indexer: a dictionary that maps ngram to an unique index
    """
    # TODO: replace with your code
    # Please DO NOT assign any ngram to index 0 which is reserved for PAD token
    #print('tokens:', tokens)

    index_list = [ngram_indexer.get(word, len(ngram_indexer.keys()) + 1) for word in tokens]
    return index_list

def process_text_dataset(dataset, n, topk=None, ngram_indexer=None):
    """
    Top level function that encodes each datum into a list of ngram indices
    @param dataset: list of Datum
    @param n: n in "n-gram"
    @param topk: #
    @param ngram_indexer: a dictionary that maps ngram to an unique index
    """
    # extract n-gram
    for i in range(len(dataset)):
        text_datum = dataset[i].raw_text
        ngrams, tokens = extract_ngram_from_text(text_datum, n)
        dataset[i].set_ngram(ngrams)
        dataset[i].set_tokens(tokens)
    # select top k ngram
    if ngram_indexer is None:
        ngram_indexer = construct_ngram_indexer([datum.ngram for datum in dataset], topk)
        #print(ngram_indexer)
    # vectorize each datum
    for i in range(len(dataset)):
        dataset[i].set_token_idx(token_to_index(dataset[i].tokens, ngram_indexer))
    return dataset, ngram_indexer

In [ ]:
# convert text data into list of index - should take few mins
# Note that we are using the train_ngram_indexer to index validation and test dataset. Why?
# Your answer:

train_data, train_ngram_indexer = process_text_dataset(trainset, ngram_n, vocab_size)
validation_data, _ = process_text_dataset(valset, ngram_n, ngram_indexer=train_ngram_indexer)
test_data, _ = process_text_dataset(testset, ngram_n, ngram_indexer=train_ngram_indexer)

## Part III: Construct Input Pipeline for PyTorch

In [ ]:
class TextClassifierDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """

    def __init__(self, data_list):
        """
        @param data_list: list of Datum
        """
        self.data_list = data_list

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        token_idx, label = self.data_list[key].token_idx, self.data_list[key].label
        return (token_idx, len(token_idx)), label


def text_classifier_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all
    data have the same length
    """
    data_list = []
    label_list = []
    length_list = []
    for datum in batch:
        label_list.append(datum[1])
        length_list.append(datum[0][1])
    max_length = np.max(length_list)
    # padding
    for datum in batch:
        padded_vec = np.pad(np.array(datum[0][0]),
                                pad_width=((0,max_length-datum[0][1])),
                                mode="constant", constant_values=0)
        data_list.append(padded_vec)

    # return tuples and send them to device (e.g. cpu/gpu depending on availability)
    return torch.from_numpy(np.array(data_list)).to(device), torch.LongTensor(length_list).to(device), torch.LongTensor(label_list).to(device)

# consturct datasets
py_train = TextClassifierDataset(train_data)
py_validation = TextClassifierDataset(validation_data)
py_test = TextClassifierDataset(test_data)

# construct data loader
train_loader = torch.utils.data.DataLoader(dataset=py_train,
                                           batch_size=batch_size,
                                           collate_fn=text_classifier_collate_func,
                                           shuffle=True)
validation_loader = torch.utils.data.DataLoader(dataset=py_validation,
                                           batch_size=batch_size,
                                           collate_fn=text_classifier_collate_func,
                                           shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=py_test,
                                           batch_size=batch_size,
                                           collate_fn=text_classifier_collate_func,
                                           shuffle=False)

In [ ]:
print("This is an training sample: {0}".format(py_train[0][0]))
print("This is a label: {0}".format(py_train[0][1]))

This is an training sample: ([1254, 1626, 357, 1824, 1621], 5)
This is a label: 21


## Part IV: Define Model [15 pts]



In [ ]:
from torch.nn import functional as F

class LogisticRegression(torch.nn.Module):
  def __init__(self, vocab_size, num_class):
    super().__init__()
    # TODO: Your code
    # Hint: Use nn.Embedding to store the feature weights, and set embedding_dim = num_class
    # Note that the # of inputs dimension for embedding shall be vocab_size+1, why?
    # In the embedding, you need to set the padding_idx argument.
    # Please see http://pytorch.org/docs/master/nn.html
    self._embed = nn.Embedding(num_embeddings = vocab_size + 1, embedding_dim = num_class, padding_idx = 0)

  def forward(self, data, length):
    """
        @param data: matrix of size (batch_size, max_length). Each row in data represents a
            review that is represented using a list of n-gram indices.
            In a batch, all samples are padded to have the same length.
    """
    # TODO: Your code
    # You need to calculate the sum of the activated feature weights
    # The output of this function should be a Tensor of shape = (batch_size, num_class), i.e
    # a batch of logits per class (No need to pass logits into Softmax here, as torch.nn.CrossEntropyLoss() takes in logits)
    #print(data.shape)
    output = self._embed(data)
    #print(output.shape)
    #output = F.sigmoid(output)
    output = torch.sum(output, dim = 1)
    #print(output.shape)
    return output

# instantiate a LR model
model = LogisticRegression(vocab_size, num_class)
# send the model to device (cpu/gpu)
model.to(device)

LogisticRegression(
  (_embed): Embedding(64001, 77, padding_idx=0)
)

## Part V: Define Loss Function and Optimizer

In [ ]:
# Loss and Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
print(learning_rate)

0.1


## Part VI: Train and Test the Model

In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter
import os

logs_base_dir = "runs"
os.makedirs(logs_base_dir, exist_ok=True)

writer = SummaryWriter()

%reload_ext tensorboard
%tensorboard --logdir "runs"

Reusing TensorBoard on port 6006 (pid 1831), started 1:24:46 ago. (Use '!kill 1831' to kill it.)

In [ ]:
import sklearn
import sklearn.metrics

# Define the test model function
def test_model(loader, model):
    """
    Help function that tests the model's performance on a dataset
    @param: loader - data loader for the dataset to test against
    """
    # set model to eval mode (no randomness (e.g. dropout) is introduced during feedforward)
    model.eval()
    all_preds = []
    all_labels = []
    for data, lengths, labels in loader:
        # should not compute gradient during eval to speed-up computation and reduce memory comsumption
        with torch.no_grad():
          logits = model(data, lengths)

        all_preds.append(logits.detach().cpu().numpy())
        all_labels.append(labels.detach().cpu().numpy())

    all_labels = np.concatenate(all_labels)
    all_logits = np.concatenate(all_preds)
    all_pred_ids = np.argmax(all_logits, axis=-1)

    # calculate metrics
    accuracy = np.sum(all_pred_ids == all_labels) / len(all_labels)
    precision = sklearn.metrics.precision_score(all_labels, all_pred_ids, average="macro")
    recall = sklearn.metrics.recall_score(all_labels, all_pred_ids, average = "macro")
    f1 = 2.0 * precision * recall / (precision + recall)

    # set model back to train model
    model.train()

    return accuracy, precision, recall, f1

In [ ]:
# print the timestamp (Please do not remove)
!date

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

# Training the Model
model.train()
global_step = 0
for epoch in tqdm(range(num_epochs)):
    for i, (data, lengths, labels) in enumerate(train_loader):
        global_step += 1

        data_batch, length_batch, label_batch = Variable(data), Variable(lengths), Variable(labels)
        outputs = model(data_batch, length_batch)
        #print(label_batch)
        #print(label_batch)
        if i == 0:
            #print(list(model.parameters()))
            pass
        loss = criterion(outputs, label_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # report performance in tensorboard
        if (i+1) % (batch_size*4) == 0:
            val_acc, val_precision, val_recall, val_f1 = test_model(validation_loader, model)
#            print('Epoch: [{0}/{1}], Step: [{2}/{3}], Loss: {4}, Validation Acc:{5}'.format(
#                   epoch+1, num_epochs, i+1, len(py_train)//batch_size, loss.data, val_acc))
            writer.add_scalar("Loss/train", loss.item(), global_step)
            writer.add_scalar("Acc/dev", val_acc, global_step)
            writer.add_scalar("Precision/dev", val_precision, global_step)
            writer.add_scalar("Recall/dev", val_recall, global_step)
            writer.add_scalar("F1/dev", val_f1, global_step)

Sun Feb 25 16:18:52 CST 2024


100%|██████████| 100/100 [02:34<00:00,  1.54s/it]


In [ ]:
# print the timestamp (Please do not remove)
!date



# Test the Model
print(test_model(test_loader, model))

Sun Feb 25 16:21:30 CST 2024
(0.8045454545454546, 0.8105016576575053, 0.8045454545454545, 0.8075125730300747)


In [ ]:
# print the timestamp (Please do not remove)
!date

state_dict = model.state_dict()
print(state_dict.keys())
torch.save(state_dict, "pytorch_model.bin")

Sun Feb 25 16:21:37 CST 2024
odict_keys(['_embed.weight'])


In [ ]:
# print the timestamp (Please do not remove)
!date

# read state dict (each parameter has a name that maps to a tensor)
state_dict = torch.load("pytorch_model.bin", map_location="cpu")

# load weights into model
model.load_state_dict(state_dict)

Sun Feb 25 16:21:50 CST 2024


<All keys matched successfully>

In [ ]:
# print the timestamp (Please do not remove)
!date

# Test the loaded model from Google Drive
print(test_model(test_loader, model))

Sun Feb 25 16:21:52 CST 2024
(0.8045454545454546, 0.8105016576575053, 0.8045454545454545, 0.8075125730300747)


## Part VII: Parameter Tuning [10 *pts*]

### Try `ngram_n=2` [5 *pts*]
Try to optimize your model F1 performance based on the validation set.
Then report your precision/recall/F1 on both validation and test sets.
What do you discover?

### Other hyper-parameter tuning [5 *pts*]
Pick one other hyper-parameter to tune on. Which one will you pick?
Then report your precision/recall/F1 on both validation and test sets.
What do you discover?

Your code & answer:

In [ ]:
# print the timestamp (please do not remove)
!date


Sun Feb 25 16:37:11 CST 2024


In **2gram.ipynb**, I modified the 1gram model to 2gram model, and the model performs worse on all the metrics (accuracy, precision, recall, f1), dropping from 0.8 to 0.7. I suspect this is because this text classification task can be done quite well by checking certain key words (i.e., 1-gram model), and applying 2-gram negatively impacts the focus on these keywords. 1-gram may be a good choice for this discriminative model, whereas high-order n-grams will certainly perform better in generative models.

In **lr_tuning.ipynb**, I modified the learning rate from 0.1 to 0.01, and the model's performance drops from 0.8 to 0.6. I conjecture this is because smaller learning rate makes the model converge slower. If the model is trained for more epochs, perhaps the model can achieve comparable performance to the one with 0.1 learning rate.

However, this conjecture is not validated, as I trained the model for 200 epochs, but the performance does not improved considerably. Therefore, there may be some other factors that spoil the model performance when learning rate is smaller.